In [143]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow.keras as keras

In [144]:
seqlen=12

In [145]:
scaler=pickle.load(P('问题2-MinMaxScaler-训练数据.csv').open('rb'))
# df=pd.read_csv('问题2-训练数据-归一化.csv',index_col='time', parse_dates=True)
df=pd.read_csv('问题2-训练数据-归一化.csv').drop(columns='time')
model=keras.models.load_model('./problem2_checkpoint.h5')

In [146]:
scaler.scale_, scaler.min_

(array([0.08025682, 0.03125977, 0.0195733 , 0.0256476 , 0.00112134,
        0.02069108]),
 array([-0.77367576, -0.92872773, -0.82912507, -0.00692485,  0.        ,
         0.48272295]))

In [147]:
df_raw=pd.read_csv('问题2-原始训练数据.csv')
df_raw

,time,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
0,2012-01-01,13.73,30.87,42.79,0.61,2.29,-22.94
1,2012-02-01,13.00,30.87,42.79,0.86,2.29,-17.86
2,2012-03-01,12.60,30.87,42.79,1.91,21.84,-7.11
3,2012-04-01,11.97,30.73,42.76,7.20,6.10,6.94
4,2012-05-01,14.18,29.99,42.58,18.10,50.80,14.61
...,...,...,...,...,...,...,...
118,2021-11-01,13.91,52.17,93.43,1.44,530.86,-5.89
119,2021-12-01,13.14,52.14,93.45,0.93,193.80,-12.41
120,2022-01-01,12.45,52.14,93.45,0.44,607.31,-17.09
121,2022-02-01,12.10,52.14,93.45,0.27,796.04,-17.93


In [148]:
pd.date_range?

In [149]:
df_test = df.iloc[-seqlen:, :]
test=df_test.values.reshape([1,seqlen,-1])
df_test.shape, test.shape

((12, 6), (1, 12, 6))

In [150]:
def predict(model, df_test: pd.DataFrame, n: int, seqlen: int):
    for i in range(n):
        test=df_test.iloc[-seqlen:, :].values.reshape([1,seqlen,-1])
        pred=model.predict(test)
        df_pred=pd.DataFrame(pred, columns=df.columns)
        df_test = pd.concat([df_test, df_pred])
        
    df_test.index=pd.Index(range(len(df_test)))
    
    return df_test

In [151]:
df_pred=predict(model, df_test, 21, seqlen=12)

In [152]:
pred=scaler.inverse_transform(df_pred)
df_pred=pd.DataFrame(data=pred, columns=df_pred.columns)
df_pred

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
0,14.180000,49.840000,81.940000,11.140000,10.670000,7.180000
1,14.770000,47.770000,79.860000,16.380000,17.780000,12.810000
2,17.660000,47.630000,76.280000,25.570000,52.070000,18.560000
3,22.100000,61.700000,85.270000,30.880000,134.370000,22.310000
4,20.360000,56.110000,85.640000,19.250000,37.340000,18.180000
5,21.010000,59.260000,87.320000,17.940000,65.530000,14.610000
6,16.930000,54.840000,92.520000,10.480000,4.830000,4.420000
7,13.910000,52.170000,93.430000,1.440000,530.860000,-5.890000
8,13.140000,52.140000,93.450000,0.930000,193.800000,-12.410000
9,12.450000,52.140000,93.450000,0.440000,607.310000,-17.090000


In [154]:
px.line(df_pred)